## Install Modules

In [1]:
# !pip install -q transformers
# !pip install -q torch
# !pip install -q datasets
# !pip install -q accelerate -U
# !pip install wandb -q
!pip install gdown

## Load Modules

In [2]:
import os
import torch
import wandb
import gdown
import logging
import numpy as np
import pandas as pd
from datetime import datetime
from datasets import Dataset
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
# from google.colab import userdata
from transformers import Trainer, TrainingArguments
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          EarlyStoppingCallback
                          )
from torch.nn import CrossEntropyLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Download Data

In [3]:
def download_drive_file_by_id(file_id, target_file_name):
  dataset_url = f"https://drive.google.com/u/1/uc?id={file_id}&export=download"
  gdown.download(dataset_url, target_file_name)
  print("File Download succesfull")

FILE_ID = ['166k7N9KV6jEDvvAr9iLTwrWyfdEcAs5p', '1-2TjS6xPfjWj9YaJGSf-JXXXfNz-2pNT', '1-1k1yHOGP7Wij1mUG2iKaSTN8i1WUgPz']
FILENAME = ["train.csv", "val_tweet.csv", "val_label.csv"]

for file_id, file_name in zip(FILE_ID, FILENAME):
    if not os.path.exists(file_name):
        download_drive_file_by_id(file_id, file_name)
    else:
        print(f"File '{file_name}' already exists.")

Downloading...
From: https://drive.google.com/u/1/uc?id=166k7N9KV6jEDvvAr9iLTwrWyfdEcAs5p&export=download
To: /kaggle/working/train.csv
100%|██████████| 914k/914k [00:00<00:00, 101MB/s]


File Download succesfull


Downloading...
From: https://drive.google.com/u/1/uc?id=1-2TjS6xPfjWj9YaJGSf-JXXXfNz-2pNT&export=download
To: /kaggle/working/val_tweet.csv
100%|██████████| 196k/196k [00:00<00:00, 60.8MB/s]


File Download succesfull


Downloading...
From: https://drive.google.com/u/1/uc?id=1-1k1yHOGP7Wij1mUG2iKaSTN8i1WUgPz&export=download
To: /kaggle/working/val_label.csv
100%|██████████| 3.80k/3.80k [00:00<00:00, 5.55MB/s]

File Download succesfull


## Load Data

In [4]:
COL_NAMES = ['index', 'text', 'label']
train_df = pd.read_csv(FILENAME[0], header=0, names=COL_NAMES)
valid_df_tweet = pd.read_csv(FILENAME[1])
valid_df_label = pd.read_csv(FILENAME[2])
valid_df = pd.merge(valid_df_tweet, valid_df_label, on='index')
valid_df.columns = COL_NAMES
train_df.drop('index', axis=1, inplace=True)
valid_df.drop('index', axis=1, inplace=True)

In [5]:
train_df.head()

,text,label
0,@me_sherya सक्छ। यो जाबो एमाले त मलाई मनै पर्दैन।,1
1,‘धार्मिक भावनाएँ भड़का कर दंगे करवाना चाहते है...,0
2,@belakoboli ये ट्वाके अङ्कल ओलि देउवा माकुने ...,0
3,"@suvashsanatani यिनिहरुले बाटो, पुल, स्कुल, बि...",2
4,हिजो पार्टी मा ओलिले एकलौटि गर्‍यो भनेर स्याल ...,0


In [6]:
valid_df.head()

,text,label
0,भोट त लौरोमा हाल्ने हो रुखमा त मल हाले हुन्छ ।,1
1,केन्द्रीय गृह एवं सहकारिता मंत्री अमित शाह (@A...,0
2,@Alive_Aleeza @bishwaprakash77 @kpsharmaoli वि...,0
3,"@cmprachanda, @ncp_madhavnepal , @SherBDeuba ,...",0
4,महाविद्ध्वान खड्ग प्रसादलाई भोट दिने भनेको जोक...,0


## Set Seed

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)

## Tokenizer Functions

In [8]:
def tokenize_function(examples):
  return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=169)

## Create Dataset Class


In [9]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

## Tokenizer and Dataset

In [10]:
from transformers import AutoTokenizer
model_name = "google/muril-large-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2214 [00:00<?, ? examples/s]

Map:   0%|          | 0/474 [00:00<?, ? examples/s]

## Model

In [11]:

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model

pytorch_model.bin:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((

## WandB Config

In [14]:
def parse_today_date():
    return f"{datetime.now().strftime('%Y-%m-%d')}"

run_name = f'model-{model_name}-{parse_today_date()}-focal-loss-no-preprocessed'.replace('/','-')
output_dir=f"./{run_name}_results"

# change this accrodiang to your env i.e google colab, kaggle, sagemakers studio
# #############################################################################

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

os.environ['WANDB_API_KEY'] = WANDB_API_KEY
os.environ["WANDB_DISABLED"] = "false"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb.init(project="NLP CHIPSAL", name=run_name)
wandb.run.name = run_name


wandb: Currently logged in as: suman-805522 (suman-smstu). Use `wandb login --relogin` to force relogin


## Trainer API Categorical Cross Entropy Loss

In [17]:
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,  # Adjusted number of epochs
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=1,
    save_strategy="steps",
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=50,
    report_to="wandb",  # Enable reporting to WandB
    run_name=run_name,
    logging_first_step=True,
    seed=42
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset
)

# Call contiguous() on the model parameters to ensure they are contiguous in memory
for name, param in model.named_parameters():
  if not param.data.is_contiguous():
    param.data = param.data.contiguous()

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/tmp/ipykernel_36/633149242.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=torch.tensor(self.alpha, device=device))


Step,Training Loss


wandb: Adding directory to artifact (./results/checkpoint-278)... Done. 44.4s


TrainOutput(global_step=278, training_loss=1.8724824095801484, metrics={'train_runtime': 415.9024, 'train_samples_per_second': 10.647, 'train_steps_per_second': 0.668, 'total_flos': 1362102377597208.0, 'train_loss': 1.8724824095801484, 'epoch': 2.0})

## Predictions and Evaluations

In [18]:
# Predictions and evaluation
y_true = valid_dataset['label']
predictions = trainer.predict(valid_dataset)
y_hat = np.argmax(predictions.predictions, axis=1)

# Save metrics to CSV
metrics_df = pd.DataFrame({'y_true': y_true, 'y_hat': y_hat})
metrics_df.to_csv(f'muril_metrics_preprocessed.csv', index=False)
logging.info("Metrics saved to metrics.csv")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/tmp/ipykernel_36/633149242.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=torch.tensor(self.alpha, device=device))


In [19]:
# Save classification report
report = classification_report(y_true, y_hat, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(f'muril_classification_report_preprocessed.csv')
logging.info("Classification report saved to classification_report.csv")

# Log GPU memory usage
logging.info(f"Peak GPU memory usage: {torch.cuda.max_memory_allocated() / (1024 ** 3):.2f} GB")

# End the WandB run
wandb.finish()

test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁
train/global_step,▁▁
test/loss,1.55943
test/runtime,14.4627
test/samples_per_second,32.774
test/steps_per_second,2.074
total_flos,1362102377597208.0
